# Forms

Django has support for designing forms containing user input.  Let's make a start by creating a new project:

In [ ]:
mkdir workspace3
django-admin startproject myproject workspace3
cd workspace3
tree .

Now create our webapp:

In [ ]:
python manage.py startapp forms
tree -I __pycache__ .

Register our app

In [ ]:
sed -i -e "/INSTALLED_APPS/ a \    'forms'," myproject/settings.py
sed -n '/^INSTALLED_APPS/,/^]/p' myproject/settings.py

We begin by defining a form:

In [ ]:
cat << EOF > forms/forms.py
from django import forms

# creating a form 
class InputForm(forms.Form):
    first_name = forms.CharField(max_length = 200, initial="Chris")
    last_name = forms.CharField(max_length = 200, initial="Seddon")
    city = forms.CharField(max_length = 200, initial="London")
    country = forms.CharField(max_length = 200, initial="UK")
    roll_number = forms.IntegerField(
		help_text = "Enter 6 digit roll number",
        initial=523681
	)
EOF

Now create our view:

In [ ]:
cat << EOF > forms/views.py
from django.http import HttpResponse
from django.shortcuts import render
from .forms import InputForm

# Create your views here.
def homeView(request):
    context = {}
    context['form']= InputForm()
    return render(request, "home.html", context)

def printResults(request):
    context = {
        'first_name':  request.POST.get('first_name'), 
        'last_name':   request.POST.get('last_name'),
        'city':        request.POST.get('city'),
        'country':     request.POST.get('country'),
        'roll_number': request.POST.get('roll_number'),
    }
    return render(request, "mytemplate.html", context)
EOF

Now set up urls:

In [ ]:
cat << EOF > forms/urls.py
from django.urls import path

from . import views

urlpatterns = [
    # called from the template: action = /forms/printResults/'
    path('printResults/', views.printResults, name='printResults'),

    # firefox 'http://localhost:7000/forms/home/'
    path('home/', views.homeView, name='homeView'),
]
EOF

Update the project urls.

In [ ]:
cat << EOF > myproject/urls.py
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('forms/', include('forms.urls')),
]
EOF

The template for the form.  Note you need to use a Cross-site request forgery token to prove the target of the POST is genuine.

In [ ]:
mkdir -p forms/templates
cat << EOF > forms/templates/home.html
<!DOCTYPE html>
<html>
<head>
<style>
body {background-color: powderblue;}
form   {color: blue;}
tr     {background-color: red;}
</style>
</head>
<body>
<form action = "/forms/printResults/" method = "post">
    {% csrf_token %}
    {{form }}
	<input type="submit" value=Submit>
</form>
</body>
EOF

The template for results:

In [ ]:
cat << EOF > forms/templates/mytemplate.html
<!DOCTYPE html>
<html>
<head>
<style>
body {background-color: powderblue;}
table, th, td {
  border: 1px solid;
}
td    {color: red;}
</style>
</head>

<h1>Results</h1>

<table>
    <tr><td>First Name</td>  <td>{{ first_name }}</td></tr>
    <tr><td>Last Name</td>   <td>{{ last_name }}</td></tr>
    <tr><td>City</td>        <td>{{ city }}</td></tr>
    <tr><td>Country</td>     <td>{{ country }}</td></tr>
    <tr><td>Roll Number</td> <td>{{ roll_number }}</td></tr>
</table>
EOF

The file structure is now:

In [ ]:
tree -I __pycache__ .

Perform migrations

In [ ]:
python manage.py migrate

Start the server

In [ ]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

Use 'touch' to get Django to reload the view.  If we don't do this the template engine gets confused and will fail to locate our templates.

In [ ]:
touch forms/views.py

Fire up `firefox` to see the app:

In [ ]:
firefox http://localhost:7000/forms/home

clean up:
* kill the server
* remove workspace

In [ ]:
cd ..
fuser -k 7000/tcp
rm -r workspace3